# Import

In [1]:
#!pip install owlready2

In [2]:
from owlready2 import *
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


# Load dataset

In [3]:
menu_dir = 'https://raw.githubusercontent.com/regenesis90/coursework/main/ontology_engineering/Dataset/Starbucks_Recent_Menus_Tall.csv'

In [4]:
starbucks_info = pd.read_csv(menu_dir)

In [5]:
product_dir = 'https://raw.githubusercontent.com/regenesis90/coursework/main/ontology_engineering/Dataset/product_category.csv'

In [6]:
product_info = pd.read_csv(product_dir)
product_info.head(3)

,Category_ID,Category_Name,Temperature,Product_ID,Product_Name
0,Coffee,Coffee,Hot,Brewed_Coffee_Dark_Roast,Brewed Coffee - Dark Roast
1,Coffee,Coffee,Hot,Brewed_Coffee_Decaf_Pike_Place_Roast,Brewed Coffee - Decaf Pike Place Roast
2,Coffee,Coffee,Hot,Brewed_Coffee_Medium_Roast,Brewed Coffee - Medium Roast


In [7]:
addons_dir = 'https://raw.githubusercontent.com/regenesis90/coursework/main/ontology_engineering/Dataset/AddOns.csv'

In [8]:
addons_info = pd.read_csv(addons_dir)
addons_info.head(3)

,ID,AddOns,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Cholesterol,Sodium,Total_Carbohydrates,Dietary_Fiber,Sugar,Protein,Vitamin_A,Vitamin_C,Calcium,Iron,Caffeine
0,Sweetened_Whipped_Cream,Sweetened Whipped Cream,60,6.0,4.0,0.2,20,5,2,0,1,0.3,6,0,0,0,0
1,Sweetened_Whipped_Cream,Sweetened Whipped Cream,80,8.0,5.0,0.2,30,10,2,0,2,0.4,10,0,2,0,0
2,Flavoured_Syrup,Flavoured Syrup,20,0.0,0.0,0.0,0,0,5,0,5,0.0,0,0,0,0,0


# Develop Ontology

## Create New Ontology
* 새로운 온톨로지 생성(빈 온톨로지)

In [9]:
ontology_name = 'http://test.org/starbucks.owl'
onto = get_ontology(ontology_name)

## Class & Properties 생성

### Class 선언

In [10]:
# class 선언
with onto:
    class Category(Thing):
        pass
    
    class Beverage(Thing):
        pass
    
    class Product(Thing):
        pass
    
    class AddOns(Thing):
        pass

### Property 선언

In [11]:
### Category가 가진 프로퍼티
with onto:
    class has_Category_Name(DataProperty):
        domain = [Category]
        range = [str]
        
    class has_Product(ObjectProperty):
        domain = [Category]
        range = [Product]

In [12]:
### Product가 가진 프로퍼티
with onto:
    class has_Temperature(DataProperty):
        domain = [Product]
        range = [Or(['Hot', 'Ice'])]
    
    class has_Category(ObjectProperty): # Product는 Category에 속함. has_Product 되어있음
        domain = [Product]
        range = [Category]
        inverse_property = has_Product 
    
    class has_Product_Name(DataProperty):
        domain = [Product]
        range = [str]
        
    class contain(ObjectProperty): # Product는 여러개의 Beverage를 포함하고 있으니깐
        domain = [Product]
        range = [Beverage]

In [13]:
# Beverage properties
with onto:
    #### 밀크, 휩, 카테고리, 애드온 ####
    
    class has_AddOns_name(DataProperty):
        domain = [AddOns]
        range = [str]
    
    class has_Beverage_name(DataProperty):
        domain = [Beverage]
        range = [str]
    
    class has_Milk(DataProperty):
        domain = [Beverage]
        range = [str]
        
    class has_Whip(DataProperty):
        domain = [Beverage]
        range = [str]
        
    class is_contained(ObjectProperty): # Beverage는 Product에 contained 됨
        domain = [Beverage]
        range = [Product]
        inverse_property = contain
        
    class has_AddOns(ObjectProperty):
        domain = [Beverage]
        range = [AddOns]
        
    #### 영양분 ####
    class has_Calories(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]

    class has_Total_Fat(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Saturated_Fat(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Trans_Fat(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Cholesterol(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Sodium(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Total_Carbohydrates(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Dietary_Fiber(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Sugar(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Protein(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Vitamin_A(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Vitamin_C(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Calcium(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Iron(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]
        
    class has_Caffeine(DataProperty):
        domain = [Or([Beverage, AddOns])]
        range = [float]

## 개별 데이터 투입

### 함수 정의

In [14]:
def create_Beverage(ID:str, name:str, Product_ID:str, Milk:str, Whip:str, Calories:float, Total_Fat:float, Saturated_Fat:float, Trans_Fat:float, Cholesterol:float, Sodium:float, Total_Carbohydrates:float, Dietary_Fiber:float, Sugar:float, Protein:float, Vitamin_A:float, Vitamin_C:float, Calcium:float, Iron:float, Caffeine:float):
    """음료 정보 투입 함수"""
    
    new_Beverage = Beverage(ID)
    new_Beverage.has_Beverage_name.append(name)
    
    if pd.isna(Milk) == False: # milk 값이 있으면
        new_Beverage.has_Milk.append(Milk)
    
    if pd.isna(Whip) == False: # Whip 값이 있으면
        new_Beverage.has_Whip.append(Whip)
    
    new_Beverage.has_Calories.append(Calories)
    new_Beverage.has_Total_Fat.append(Total_Fat)
    new_Beverage.has_Saturated_Fat.append(Saturated_Fat)
    new_Beverage.has_Trans_Fat.append(Trans_Fat)
    new_Beverage.has_Cholesterol.append(Cholesterol)
    new_Beverage.has_Sodium.append(Sodium)
    new_Beverage.has_Total_Carbohydrates.append(Total_Carbohydrates)
    new_Beverage.has_Dietary_Fiber.append(Dietary_Fiber)
    new_Beverage.has_Sugar.append(Sugar)
    new_Beverage.has_Protein.append(Protein)
    new_Beverage.has_Vitamin_A.append(Vitamin_A)
    new_Beverage.has_Vitamin_C.append(Vitamin_C)
    new_Beverage.has_Calcium.append(Calcium)
    new_Beverage.has_Iron.append(Iron)
    new_Beverage.has_Caffeine.append(Caffeine)
    
    new_Beverage.is_contained.append(Product(Product_ID))
    
    return new_Beverage

In [15]:
def create_Product(Product_ID, Product_Name, Category_ID, Temperature):
    """프로덕트 정보 투입 함수"""
    
    new_Product = Product(Product_ID)
    
    new_Product.has_Product_Name.append(Product_Name)
    new_Product.has_Temperature.append(Temperature)
    
    new_Product.has_Category.append(Category(Category_ID))
    
    return new_Product

In [16]:
def create_Category(Category_ID, Category_Name):
    
    new_Category = Category(Category_ID)
    new_Category.has_Category_Name.append(Category_Name)
    
    return new_Category

In [17]:
def create_AddOns(ID:str, name:str, Calories, Total_Fat, Saturated_Fat, Trans_Fat, Cholesterol, Sodium, Total_Carbohydrates, Dietary_Fiber, Sugar, Protein, Vitamin_A, Vitamin_C, Calcium, Iron, Caffeine):
    """애드온 정보 투입 함수"""
    
    new_AddOns = AddOns(ID)
    new_AddOns.has_AddOns_name.append(name)
    
    new_AddOns.has_Calories.append(Calories)
    new_AddOns.has_Total_Fat.append(Total_Fat)
    new_AddOns.has_Saturated_Fat.append(Saturated_Fat)
    new_AddOns.has_Trans_Fat.append(Trans_Fat)
    new_AddOns.has_Cholesterol.append(Cholesterol)
    new_AddOns.has_Sodium.append(Sodium)
    new_AddOns.has_Total_Carbohydrates.append(Total_Carbohydrates)
    new_AddOns.has_Dietary_Fiber.append(Dietary_Fiber)
    new_AddOns.has_Sugar.append(Sugar)
    new_AddOns.has_Protein.append(Protein)
    new_AddOns.has_Vitamin_A.append(Vitamin_A)
    new_AddOns.has_Vitamin_C.append(Vitamin_C)
    new_AddOns.has_Calcium.append(Calcium)
    new_AddOns.has_Iron.append(Iron)
    new_AddOns.has_Caffeine.append(Caffeine)
    
    return new_AddOns

### 정보 투입

In [18]:
# 카테고리 정보 넣기
category_info = product_info[['Category_ID', 'Category_Name']].drop_duplicates()

for i in tqdm(range(len(category_info))):
    
    ca_ID = str(category_info['Category_ID'].iloc[i])
    ca_Name = str(category_info['Category_Name'].iloc[i])
    
    new_Category = create_Category(ca_ID, ca_Name)

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 4009.37it/s]


In [19]:
product_info

,Category_ID,Category_Name,Temperature,Product_ID,Product_Name
0,Coffee,Coffee,Hot,Brewed_Coffee_Dark_Roast,Brewed Coffee - Dark Roast
1,Coffee,Coffee,Hot,Brewed_Coffee_Decaf_Pike_Place_Roast,Brewed Coffee - Decaf Pike Place Roast
2,Coffee,Coffee,Hot,Brewed_Coffee_Medium_Roast,Brewed Coffee - Medium Roast
3,Coffee,Coffee,Hot,Brewed_Coffee_True_North_Blend_Blonde_Roast,Brewed Coffee - True North Blend Blonde Roast
4,Coffee,Coffee,Hot,Caffe_Misto,Caffe Misto
...,...,...,...,...,...
84,Hot_Chocolate_and_Other,Hot Chocolate & Other,Hot,Skinny_Hot_Chocolate,Skinny Hot Chocolate
85,Hot_Chocolate_and_Other,Hot Chocolate & Other,Hot,Steamed_Apple_Juice,Steamed Apple Juice
86,Hot_Chocolate_and_Other,Hot Chocolate & Other,Hot,Steamed_Milk,Steamed Milk
87,Hot_Chocolate_and_Other,Hot Chocolate & Other,Hot,Vanilla_Steamer,Vanilla Steamer


In [20]:
# 프로덕트 정보 넣기
for i in tqdm(range(len(product_info))):
    
    pr_ID = str(product_info['Product_ID'].iloc[i])
    pr_Product_Name = str(product_info['Product_Name'].iloc[i])
    pr_Category = str(product_info['Category_ID'].iloc[i])
    pr_Temperature = str(product_info['Temperature'].iloc[i])
    
    new_Product = create_Product(pr_ID, pr_Product_Name, pr_Category, pr_Temperature)

100%|████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 3432.64it/s]


In [21]:
# Beverage 정보 넣기
for i in tqdm(range(len(starbucks_info))):
    
    cur_ID = str(starbucks_info['ID'].iloc[i])
    cur_name = str(starbucks_info['Beverage'].iloc[i])
    cur_Product_ID = starbucks_info['Product_ID'].iloc[i]
    
    cur_Milk = str(starbucks_info['Milk'].iloc[i])
    cur_Whip = str(starbucks_info['Whip'].iloc[i])
    
    cur_Calories = float(starbucks_info['Calories'].iloc[i])
    cur_Total_Fat = float(starbucks_info['Total_Fat'].iloc[i])
    cur_Saturated_Fat = float(starbucks_info['Saturated_Fat'].iloc[i])
    cur_Trans_Fat = float(starbucks_info['Trans_Fat'].iloc[i])
    cur_Cholesterol = float(starbucks_info['Cholesterol'].iloc[i])
    
    cur_Sodium = float(starbucks_info['Sodium'].iloc[i])
    cur_Total_Carbohydrates = float(starbucks_info['Total_Carbohydrates'].iloc[i])
    cur_Dietary_Fiber = float(starbucks_info['Dietary_Fiber'].iloc[i])
    cur_Sugar = float(starbucks_info['Sugar'].iloc[i])
    cur_Protein = float(starbucks_info['Protein'].iloc[i])
    
    cur_Vitamin_A = float(starbucks_info['Vitamin_A'].iloc[i])
    cur_Vitamin_C = float(starbucks_info['Vitamin_C'].iloc[i])
    cur_Calcium = float(starbucks_info['Calcium'].iloc[i])
    cur_Iron = float(starbucks_info['Iron'].iloc[i])
    cur_Caffeine = float(starbucks_info['Caffeine'].iloc[i])
    
    new_Beverage = create_Beverage(cur_ID, cur_name, cur_Product_ID, cur_Milk, cur_Whip, 
                                   cur_Calories, cur_Total_Fat, cur_Saturated_Fat, cur_Trans_Fat, cur_Cholesterol, 
                                   cur_Sodium, cur_Total_Carbohydrates, cur_Dietary_Fiber, cur_Sugar, cur_Protein, cur_Vitamin_A, cur_Vitamin_C, cur_Calcium, cur_Iron, cur_Caffeine)

100%|██████████████████████████████████████████████████████████████████████████████| 335/335 [00:00<00:00, 1329.13it/s]


In [22]:
# AddOns 정보 넣기
for i in tqdm(range(len(addons_info))):
    
    ad_ID = str(addons_info['ID'].iloc[i])
    ad_name = str(addons_info['AddOns'].iloc[i])
    
    ad_Calories = float(addons_info['Calories'].iloc[i])
    ad_Total_Fat = float(addons_info['Total_Fat'].iloc[i])
    ad_Saturated_Fat = float(addons_info['Saturated_Fat'].iloc[i])
    ad_Trans_Fat = float(addons_info['Trans_Fat'].iloc[i])
    ad_Cholesterol = float(addons_info['Cholesterol'].iloc[i])
    
    ad_Sodium = float(addons_info['Sodium'].iloc[i])
    ad_Total_Carbohydrates = float(addons_info['Total_Carbohydrates'].iloc[i])
    ad_Dietary_Fiber = float(addons_info['Dietary_Fiber'].iloc[i])
    ad_Sugar = float(addons_info['Sugar'].iloc[i])
    ad_Protein = float(addons_info['Protein'].iloc[i])
    
    ad_Vitamin_A = float(addons_info['Vitamin_A'].iloc[i])
    ad_Vitamin_C = float(addons_info['Vitamin_C'].iloc[i])
    ad_Calcium = float(addons_info['Calcium'].iloc[i])
    ad_Iron = float(addons_info['Iron'].iloc[i])
    ad_Caffeine = float(addons_info['Caffeine'].iloc[i])
    
    new_AddOns = create_AddOns(ad_ID, ad_name,
                               ad_Calories, ad_Total_Fat, ad_Saturated_Fat, ad_Trans_Fat, ad_Cholesterol,
                               ad_Sodium, ad_Total_Carbohydrates, ad_Dietary_Fiber, ad_Sugar, ad_Protein,
                               ad_Vitamin_A, ad_Vitamin_C, ad_Calcium, ad_Iron, ad_Caffeine)

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1114.17it/s]


## 구축 온톨로지 저장

In [23]:
onto.save(file = 'ontology_starbucks.rdf', format = 'rdfxml')

# 온톨로지 내용 확인

## Inverse Properties

In [24]:
Beverage('Brewed_Coffee_True_North_Blend_Blonde_Roast').has_Calories

[4.0]